# 對抗攻擊 (Adversarial Attack)

## 學習目標
- 理解對抗樣本的概念
- 實作 FGSM 攻擊
- 實作 PGD 迭代攻擊
- 學習對抗防禦策略

## 對應課程
- [李宏毅 ML 2021 - Adversarial Attack](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW10: Adversarial Attack](https://github.com/ga642381/ML2021-Spring/tree/main/HW10)

## 對抗攻擊概述

```
對抗攻擊分類
├── 按目標
│   ├── Non-targeted: 讓模型預測錯誤
│   └── Targeted: 讓模型預測為指定類別
│
├── 按知識
│   ├── White-box: 完全了解模型
│   └── Black-box: 只能查詢模型
│
└── 按擾動
    ├── L∞: 每個像素最大改變量
    ├── L2: 擾動的歐幾里得範數
    └── L0: 改變的像素數量
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# ========== 準備模型和資料 ==========

# 載入預訓練模型
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model = model.to(device)
model.eval()

# 預處理
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# ImageNet 正規化參數
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)

def normalize(x):
    return (x - mean) / std

def denormalize(x):
    return x * std + mean

print("模型載入完成")

In [ ]:
# ========== 建立範例圖片 ==========

def create_test_image():
    """
    建立測試圖片
    """
    # 建立一個有特徵的圖片
    img = np.zeros((224, 224, 3), dtype=np.uint8)
    
    # 添加一些形狀和顏色
    for i in range(224):
        for j in range(224):
            # 圓形物體
            if (i - 112)**2 + (j - 112)**2 < 60**2:
                img[i, j] = [255, 180, 100]
            # 背景漸層
            else:
                img[i, j] = [100 + i // 3, 150, 200 - j // 3]
    
    return Image.fromarray(img)

# 建立圖片
original_image = create_test_image()
original_tensor = preprocess(original_image).unsqueeze(0).to(device)

# 原始預測
with torch.no_grad():
    output = model(normalize(original_tensor))
    original_pred = output.argmax(dim=1).item()
    original_conf = F.softmax(output, dim=1)[0, original_pred].item()

print(f"原始預測類別: {original_pred}")
print(f"原始信心度: {original_conf:.4f}")

plt.figure(figsize=(4, 4))
plt.imshow(original_image)
plt.title(f'Original (Class: {original_pred}, Conf: {original_conf:.2%})')
plt.axis('off')
plt.show()

---
## Part 1: FGSM (Fast Gradient Sign Method)

FGSM 是最簡單的對抗攻擊方法，只需要一步梯度計算。

### 公式

**Non-targeted Attack:**
$$x_{adv} = x + \epsilon \cdot \text{sign}(\nabla_x L(f(x), y))$$

**Targeted Attack:**
$$x_{adv} = x - \epsilon \cdot \text{sign}(\nabla_x L(f(x), y_{target}))$$

In [ ]:
# ========== FGSM 攻擊 ==========

def fgsm_attack(model, image, label, epsilon, targeted=False):
    """
    FGSM 攻擊
    
    Args:
        model: 目標模型
        image: 原始圖像 (0-1 範圍)
        label: 標籤（non-targeted: 原始標籤, targeted: 目標標籤）
        epsilon: 擾動大小
        targeted: 是否為目標攻擊
    
    Returns:
        對抗樣本
    """
    image = image.clone().detach().requires_grad_(True)
    
    # 前向傳播
    output = model(normalize(image))
    
    # 計算損失
    loss = F.cross_entropy(output, torch.tensor([label]).to(device))
    
    # 反向傳播
    model.zero_grad()
    loss.backward()
    
    # 獲取梯度符號
    grad_sign = image.grad.data.sign()
    
    # 生成對抗樣本
    if targeted:
        # 目標攻擊：減少目標類別的損失
        adv_image = image - epsilon * grad_sign
    else:
        # 非目標攻擊：增加原始類別的損失
        adv_image = image + epsilon * grad_sign
    
    # 裁剪到有效範圍
    adv_image = torch.clamp(adv_image, 0, 1)
    
    return adv_image.detach()

# 測試不同 epsilon 的效果
epsilons = [0, 0.01, 0.02, 0.05, 0.1, 0.2]

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for ax, eps in zip(axes.flat, epsilons):
    if eps == 0:
        adv_tensor = original_tensor.clone()
    else:
        adv_tensor = fgsm_attack(model, original_tensor, original_pred, eps)
    
    # 預測
    with torch.no_grad():
        output = model(normalize(adv_tensor))
        pred = output.argmax(dim=1).item()
        conf = F.softmax(output, dim=1)[0, pred].item()
    
    # 顯示
    adv_img = adv_tensor.squeeze().permute(1, 2, 0).cpu().numpy()
    ax.imshow(adv_img)
    
    color = 'green' if pred == original_pred else 'red'
    ax.set_title(f'ε={eps}\nPred: {pred} ({conf:.1%})', color=color)
    ax.axis('off')

plt.suptitle('FGSM Attack with Different Epsilon', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ========== 可視化擾動 ==========

epsilon = 0.05
adv_tensor = fgsm_attack(model, original_tensor, original_pred, epsilon)

# 計算擾動
perturbation = (adv_tensor - original_tensor).squeeze().permute(1, 2, 0).cpu().numpy()

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# 原圖
axes[0].imshow(original_tensor.squeeze().permute(1, 2, 0).cpu().numpy())
axes[0].set_title('Original')
axes[0].axis('off')

# 擾動（放大顯示）
perturbation_vis = (perturbation - perturbation.min()) / (perturbation.max() - perturbation.min())
axes[1].imshow(perturbation_vis)
axes[1].set_title(f'Perturbation (ε={epsilon})')
axes[1].axis('off')

# 放大的擾動
axes[2].imshow(perturbation * 10 + 0.5)  # 放大 10 倍
axes[2].set_title('Perturbation (10x magnified)')
axes[2].axis('off')

# 對抗樣本
with torch.no_grad():
    output = model(normalize(adv_tensor))
    pred = output.argmax(dim=1).item()
    conf = F.softmax(output, dim=1)[0, pred].item()

axes[3].imshow(adv_tensor.squeeze().permute(1, 2, 0).cpu().numpy())
axes[3].set_title(f'Adversarial (Pred: {pred}, {conf:.1%})')
axes[3].axis('off')

plt.tight_layout()
plt.show()

# 計算擾動統計
print(f"擾動統計:")
print(f"  L∞ norm: {perturbation.max():.4f}")
print(f"  L2 norm: {np.linalg.norm(perturbation):.4f}")
print(f"  Mean: {perturbation.mean():.6f}")

---
## Part 2: PGD (Projected Gradient Descent)

PGD 是迭代版本的 FGSM，通常更強大。

### 公式
$$x^{t+1} = \Pi_{x + S} \left( x^t + \alpha \cdot \text{sign}(\nabla_x L(f(x^t), y)) \right)$$

其中 $\Pi$ 是投影操作，將擾動限制在 $\epsilon$-球內。

In [ ]:
# ========== PGD 攻擊 ==========

def pgd_attack(model, image, label, epsilon, alpha, num_iter, targeted=False, random_start=True):
    """
    PGD 攻擊
    
    Args:
        model: 目標模型
        image: 原始圖像
        label: 標籤
        epsilon: 最大擾動（L∞）
        alpha: 步長
        num_iter: 迭代次數
        targeted: 是否為目標攻擊
        random_start: 是否從隨機點開始
    """
    # 初始化
    if random_start:
        # 從原始圖像的 epsilon 鄰域內隨機選擇起點
        adv_image = image + torch.empty_like(image).uniform_(-epsilon, epsilon)
        adv_image = torch.clamp(adv_image, 0, 1)
    else:
        adv_image = image.clone()
    
    for i in range(num_iter):
        adv_image = adv_image.clone().detach().requires_grad_(True)
        
        # 前向傳播
        output = model(normalize(adv_image))
        loss = F.cross_entropy(output, torch.tensor([label]).to(device))
        
        # 反向傳播
        model.zero_grad()
        loss.backward()
        
        # 更新
        grad_sign = adv_image.grad.data.sign()
        
        if targeted:
            adv_image = adv_image - alpha * grad_sign
        else:
            adv_image = adv_image + alpha * grad_sign
        
        # 投影到 epsilon-球內
        perturbation = torch.clamp(adv_image - image, -epsilon, epsilon)
        adv_image = torch.clamp(image + perturbation, 0, 1)
    
    return adv_image.detach()

# 測試 PGD
epsilon = 0.03
alpha = 0.01
num_iter = 10

pgd_adv = pgd_attack(model, original_tensor, original_pred, epsilon, alpha, num_iter)

# 比較 FGSM 和 PGD
fgsm_adv = fgsm_attack(model, original_tensor, original_pred, epsilon)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# 原圖
axes[0].imshow(original_tensor.squeeze().permute(1, 2, 0).cpu().numpy())
with torch.no_grad():
    out = model(normalize(original_tensor))
    pred = out.argmax(1).item()
    conf = F.softmax(out, 1)[0, pred].item()
axes[0].set_title(f'Original\nPred: {pred} ({conf:.1%})')
axes[0].axis('off')

# FGSM
axes[1].imshow(fgsm_adv.squeeze().permute(1, 2, 0).cpu().numpy())
with torch.no_grad():
    out = model(normalize(fgsm_adv))
    pred = out.argmax(1).item()
    conf = F.softmax(out, 1)[0, pred].item()
axes[1].set_title(f'FGSM (ε={epsilon})\nPred: {pred} ({conf:.1%})')
axes[1].axis('off')

# PGD
axes[2].imshow(pgd_adv.squeeze().permute(1, 2, 0).cpu().numpy())
with torch.no_grad():
    out = model(normalize(pgd_adv))
    pred = out.argmax(1).item()
    conf = F.softmax(out, 1)[0, pred].item()
axes[2].set_title(f'PGD ({num_iter} iter)\nPred: {pred} ({conf:.1%})')
axes[2].axis('off')

# 擾動差異
fgsm_pert = (fgsm_adv - original_tensor).abs().sum(1).squeeze().cpu().numpy()
pgd_pert = (pgd_adv - original_tensor).abs().sum(1).squeeze().cpu().numpy()
axes[3].imshow(pgd_pert - fgsm_pert, cmap='RdBu')
axes[3].set_title('PGD - FGSM Perturbation Diff')
axes[3].axis('off')

plt.tight_layout()
plt.show()

---
## Part 3: Targeted Attack

目標攻擊：讓模型預測為指定的類別。

In [ ]:
# ========== Targeted Attack ==========

def targeted_pgd_attack(model, image, target_class, epsilon, alpha, num_iter):
    """
    目標 PGD 攻擊
    """
    adv_image = image.clone()
    
    for i in range(num_iter):
        adv_image = adv_image.clone().detach().requires_grad_(True)
        
        output = model(normalize(adv_image))
        
        # 目標攻擊：最小化與目標類別的損失
        loss = F.cross_entropy(output, torch.tensor([target_class]).to(device))
        
        model.zero_grad()
        loss.backward()
        
        # 負梯度方向（減少損失）
        adv_image = adv_image - alpha * adv_image.grad.data.sign()
        
        # 投影
        perturbation = torch.clamp(adv_image - image, -epsilon, epsilon)
        adv_image = torch.clamp(image + perturbation, 0, 1)
        
        # 檢查是否成功
        with torch.no_grad():
            pred = model(normalize(adv_image)).argmax(1).item()
            if pred == target_class:
                break
    
    return adv_image.detach()

# 測試目標攻擊
target_classes = [100, 200, 300, 400, 500]

fig, axes = plt.subplots(1, len(target_classes) + 1, figsize=(18, 3))

# 原圖
axes[0].imshow(original_tensor.squeeze().permute(1, 2, 0).cpu().numpy())
with torch.no_grad():
    pred = model(normalize(original_tensor)).argmax(1).item()
axes[0].set_title(f'Original\nPred: {pred}')
axes[0].axis('off')

# 目標攻擊
for ax, target in zip(axes[1:], target_classes):
    adv = targeted_pgd_attack(model, original_tensor, target, epsilon=0.1, alpha=0.02, num_iter=50)
    
    with torch.no_grad():
        out = model(normalize(adv))
        pred = out.argmax(1).item()
        conf = F.softmax(out, 1)[0, pred].item()
    
    ax.imshow(adv.squeeze().permute(1, 2, 0).cpu().numpy())
    color = 'green' if pred == target else 'red'
    ax.set_title(f'Target: {target}\nPred: {pred} ({conf:.1%})', color=color)
    ax.axis('off')

plt.suptitle('Targeted PGD Attack', fontsize=14)
plt.tight_layout()
plt.show()

---
## Part 4: 對抗防禦

### 常見防禦方法

In [ ]:
# ========== 防禦方法 1: Input Transformation ==========

def jpeg_compression(image, quality=75):
    """
    JPEG 壓縮作為防禦
    可以去除對抗擾動
    """
    from io import BytesIO
    
    # 轉為 PIL Image
    img_np = (image.squeeze().permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    pil_img = Image.fromarray(img_np)
    
    # JPEG 壓縮
    buffer = BytesIO()
    pil_img.save(buffer, format='JPEG', quality=quality)
    buffer.seek(0)
    compressed_img = Image.open(buffer)
    
    # 轉回 tensor
    compressed_tensor = transforms.ToTensor()(compressed_img).unsqueeze(0).to(device)
    
    return compressed_tensor

def gaussian_blur(image, kernel_size=5, sigma=1.0):
    """
    高斯模糊作為防禦
    """
    from torchvision.transforms.functional import gaussian_blur as blur
    return blur(image, kernel_size, [sigma, sigma])

# 測試防禦效果
adv_image = pgd_attack(model, original_tensor, original_pred, 0.05, 0.01, 20)

# 應用防禦
jpeg_defended = jpeg_compression(adv_image, quality=50)
blur_defended = gaussian_blur(adv_image, kernel_size=5, sigma=2.0)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

images = [original_tensor, adv_image, jpeg_defended, blur_defended]
titles = ['Original', 'Adversarial', 'JPEG Defense', 'Blur Defense']

for ax, img, title in zip(axes, images, titles):
    with torch.no_grad():
        out = model(normalize(img))
        pred = out.argmax(1).item()
        conf = F.softmax(out, 1)[0, pred].item()
    
    ax.imshow(img.squeeze().permute(1, 2, 0).cpu().numpy())
    color = 'green' if pred == original_pred else 'red'
    ax.set_title(f'{title}\nPred: {pred} ({conf:.1%})', color=color)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ========== 防禦方法 2: Adversarial Training ==========

def adversarial_training_step(model, images, labels, epsilon, alpha, num_iter, optimizer):
    """
    對抗訓練的一個步驟
    
    1. 生成對抗樣本
    2. 用對抗樣本訓練模型
    """
    model.eval()
    
    # 生成對抗樣本
    adv_images = images.clone()
    for _ in range(num_iter):
        adv_images = adv_images.clone().detach().requires_grad_(True)
        
        outputs = model(adv_images)
        loss = F.cross_entropy(outputs, labels)
        
        loss.backward()
        
        adv_images = adv_images + alpha * adv_images.grad.sign()
        perturbation = torch.clamp(adv_images - images, -epsilon, epsilon)
        adv_images = torch.clamp(images + perturbation, 0, 1)
    
    # 訓練模型
    model.train()
    optimizer.zero_grad()
    
    outputs = model(adv_images.detach())
    loss = F.cross_entropy(outputs, labels)
    
    loss.backward()
    optimizer.step()
    
    return loss.item()

print("對抗訓練函數已定義")
print("\n對抗訓練流程:")
print("1. 對每個訓練批次生成對抗樣本")
print("2. 用對抗樣本更新模型")
print("3. 這使模型對對抗攻擊更魯棒")

In [ ]:
# ========== 防禦方法 3: Ensemble Defense ==========

def ensemble_predict(models, image):
    """
    集成預測
    
    多個模型投票決定最終預測
    """
    predictions = []
    
    for model in models:
        model.eval()
        with torch.no_grad():
            output = model(normalize(image))
            pred = output.argmax(1).item()
            predictions.append(pred)
    
    # 多數投票
    from collections import Counter
    vote = Counter(predictions).most_common(1)[0][0]
    
    return vote, predictions

print("集成防禦說明:")
print("- 使用多個不同的模型")
print("- 對抗樣本通常只對單一模型有效")
print("- 集成可以提高魯棒性")

---
## Part 5: 攻擊成功率評估

In [ ]:
# ========== 評估攻擊成功率 ==========

def evaluate_attack(model, images, labels, attack_fn, attack_params):
    """
    評估攻擊成功率
    
    Returns:
        success_rate: 攻擊成功率
        avg_perturbation: 平均擾動大小
    """
    model.eval()
    
    total = len(images)
    successful = 0
    perturbations = []
    
    for i, (img, label) in enumerate(zip(images, labels)):
        img = img.unsqueeze(0).to(device)
        
        # 生成對抗樣本
        adv_img = attack_fn(model, img, label, **attack_params)
        
        # 檢查是否成功
        with torch.no_grad():
            pred = model(normalize(adv_img)).argmax(1).item()
        
        if pred != label:
            successful += 1
        
        # 計算擾動
        pert = (adv_img - img).abs().max().item()
        perturbations.append(pert)
    
    success_rate = successful / total
    avg_pert = np.mean(perturbations)
    
    return success_rate, avg_pert

# 生成測試資料
test_images = [original_tensor.squeeze() for _ in range(10)]
test_labels = [original_pred for _ in range(10)]

# 評估不同攻擊
attacks = {
    'FGSM ε=0.01': {'epsilon': 0.01},
    'FGSM ε=0.05': {'epsilon': 0.05},
    'FGSM ε=0.10': {'epsilon': 0.10},
}

print("攻擊效果評估:")
print("-" * 50)
for name, params in attacks.items():
    success_rate, avg_pert = evaluate_attack(
        model, test_images, test_labels, fgsm_attack, params
    )
    print(f"{name:20s} | 成功率: {success_rate:.1%} | 平均擾動: {avg_pert:.4f}")

---
## 🎯 總結

### 攻擊方法比較

| 方法 | 優點 | 缺點 |
|------|------|------|
| **FGSM** | 快速（單次前向+反向）| 成功率較低 |
| **PGD** | 更強大、更可靠 | 計算較慢 |
| **C&W** | 產生最小擾動 | 非常慢 |

### 防禦方法

| 方法 | 原理 | 效果 |
|------|------|------|
| **Input Transform** | 去除擾動 | 有限 |
| **Adversarial Training** | 訓練時加入對抗樣本 | 最有效 |
| **Ensemble** | 多模型投票 | 增加攻擊難度 |

### 李宏毅 HW10 技巧

```
攻擊要點:
1. 使用 PGD 多次迭代
2. 適當的 epsilon 和 alpha
3. 不同的初始化策略
4. 組合多種攻擊方法

防禦要點:
1. Adversarial Training 是最有效的
2. 結合多種防禦策略
3. 考慮攻擊的遷移性
```

### 下一步

前往 `transfer_adaptation/domain_adaptation.ipynb` 學習域適應！

---
## 🏋️ 練習

### 練習 1: 比較不同攻擊方法的成功率與擾動大小

在相同的 L∞ 預算下，比較 FGSM、PGD（不同迭代次數）的攻擊成功率。

In [ ]:
# ========== 練習 1: 比較不同攻擊方法 ==========

def experiment_attack_methods(model, image, true_label, epsilon=0.05):
    """
    比較不同攻擊方法的效果
    """
    results = []
    
    # FGSM
    adv_fgsm = fgsm_attack(model, image, true_label, epsilon)
    with torch.no_grad():
        pred_fgsm = model(normalize(adv_fgsm)).argmax(1).item()
    pert_fgsm = (adv_fgsm - image).abs().max().item()
    results.append({
        'method': 'FGSM',
        'success': pred_fgsm != true_label,
        'perturbation': pert_fgsm,
        'pred': pred_fgsm
    })
    
    # PGD with different iterations
    for num_iter in [5, 10, 20, 40]:
        alpha = epsilon / 4
        adv_pgd = pgd_attack(model, image, true_label, epsilon, alpha, num_iter)
        with torch.no_grad():
            pred_pgd = model(normalize(adv_pgd)).argmax(1).item()
        pert_pgd = (adv_pgd - image).abs().max().item()
        results.append({
            'method': f'PGD-{num_iter}',
            'success': pred_pgd != true_label,
            'perturbation': pert_pgd,
            'pred': pred_pgd
        })
    
    return results

# 執行實驗
print("練習 1: 比較不同攻擊方法")
print("=" * 60)

epsilon = 0.05
results = experiment_attack_methods(model, original_tensor, original_pred, epsilon)

print(f"\n原始預測: {original_pred}")
print(f"擾動預算 (ε): {epsilon}")
print("-" * 60)
print(f"{'方法':<12} | {'攻擊成功':^10} | {'L∞ 擾動':^10} | {'新預測':^8}")
print("-" * 60)

for r in results:
    status = "✓ 成功" if r['success'] else "✗ 失敗"
    print(f"{r['method']:<12} | {status:^10} | {r['perturbation']:<10.4f} | {r['pred']:^8}")

# 視覺化
fig, axes = plt.subplots(1, len(results) + 1, figsize=(16, 3))

# 原圖
axes[0].imshow(original_tensor.squeeze().permute(1, 2, 0).cpu().numpy())
axes[0].set_title(f'Original\nPred: {original_pred}')
axes[0].axis('off')

# 各攻擊方法
for idx, r in enumerate(results):
    if r['method'] == 'FGSM':
        adv = fgsm_attack(model, original_tensor, original_pred, epsilon)
    else:
        num_iter = int(r['method'].split('-')[1])
        adv = pgd_attack(model, original_tensor, original_pred, epsilon, epsilon/4, num_iter)
    
    axes[idx + 1].imshow(adv.squeeze().permute(1, 2, 0).cpu().numpy())
    color = 'red' if r['success'] else 'green'
    axes[idx + 1].set_title(f"{r['method']}\nPred: {r['pred']}", color=color)
    axes[idx + 1].axis('off')

plt.suptitle(f'Attack Comparison (ε={epsilon})', fontsize=12)
plt.tight_layout()
plt.show()

print("\n結論：")
print("- FGSM 是最快的但可能成功率較低")
print("- PGD 隨著迭代次數增加通常更有效")
print("- 但迭代過多可能收益遞減")

### 練習 2: 比較不同防禦方法的效果

測試 JPEG 壓縮、高斯模糊等輸入變換防禦方法在不同攻擊強度下的效果。

In [ ]:
# ========== 練習 2: 比較不同防禦方法 ==========

def experiment_defense_methods(model, original_image, original_label, epsilons=[0.02, 0.05, 0.1]):
    """
    比較不同防禦方法在各種攻擊強度下的效果
    """
    defense_methods = {
        'No Defense': lambda x: x,
        'JPEG-75': lambda x: jpeg_compression(x, quality=75),
        'JPEG-50': lambda x: jpeg_compression(x, quality=50),
        'Blur-3': lambda x: gaussian_blur(x, kernel_size=3, sigma=1.0),
        'Blur-5': lambda x: gaussian_blur(x, kernel_size=5, sigma=2.0),
    }
    
    results = {method: [] for method in defense_methods}
    
    for eps in epsilons:
        # 生成對抗樣本
        adv_image = pgd_attack(model, original_image, original_label, eps, eps/4, 20)
        
        for method_name, defense_fn in defense_methods.items():
            defended = defense_fn(adv_image)
            
            with torch.no_grad():
                pred = model(normalize(defended)).argmax(1).item()
            
            # 記錄是否成功防禦（恢復原預測）
            success = (pred == original_label)
            results[method_name].append(success)
    
    return results, epsilons

# 執行實驗
print("練習 2: 比較不同防禦方法")
print("=" * 60)

epsilons = [0.02, 0.05, 0.1, 0.15]
results, eps_list = experiment_defense_methods(model, original_tensor, original_pred, epsilons)

# 繪製結果
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(eps_list))
width = 0.15
multiplier = 0

colors = ['#e74c3c', '#3498db', '#2980b9', '#27ae60', '#1abc9c']

for idx, (method, successes) in enumerate(results.items()):
    offset = width * multiplier
    success_rates = [int(s) for s in successes]
    bars = ax.bar(x + offset, success_rates, width, label=method, color=colors[idx])
    multiplier += 1

ax.set_xlabel('攻擊強度 (ε)')
ax.set_ylabel('防禦成功 (1=成功恢復原預測)')
ax.set_title('不同防禦方法在各攻擊強度下的表現')
ax.set_xticks(x + width * 2)
ax.set_xticklabels([f'ε={e}' for e in eps_list])
ax.legend(loc='upper right')
ax.set_ylim(0, 1.2)
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

# 打印總結
print("\n防禦方法總結:")
print("-" * 60)
print(f"{'方法':<15} | " + " | ".join([f'ε={e}' for e in eps_list]))
print("-" * 60)
for method, successes in results.items():
    success_str = " | ".join(['✓' if s else '✗' for s in successes])
    print(f"{method:<15} | {success_str}")

print("\n結論：")
print("- 較強的防禦（如高壓縮 JPEG、大核模糊）對強攻擊更有效")
print("- 但過度防禦可能影響乾淨圖像的準確率")
print("- 需要在安全性和準確率間取得平衡")